In [28]:
import pandas as pd
from aerobot.io import load_training_data, load_validation_data, asset_path
from aerobot.utls import process_data
from sklearn.linear_model import LogisticRegression
from aerobot.models import GeneralClassifier


# initialize accuray storage container
accuracies = {"feature_set":[], "accuracy":[],"balanced_accuracy":[],"test_accuracy":[],"test_balanced_accuracy":[]}

print("running on number of genes")
features = ['number_of_genes']
feature_type = "metadata"
training_data = load_training_data(feature_type=feature_type)
validation_data = load_validation_data(feature_type=feature_type)
training_data["features"] = training_data["features"][features]
validation_data["features"] = validation_data["features"][features]
cleaned_data = process_data(training_data["features"], training_data["labels"]["physiology"], validation_data["features"], validation_data["labels"]["physiology"])

model = GeneralClassifier(model_class=LogisticRegression, params={'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100000, 'C': 1})

# Fit the classifier with optimized parameters
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
# balanced accuracy
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])
#print accuracy and balanced accuracy

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])

# store accuracies in a dict for analysis later
accuracies["feature_set"].append("Number of genes")
accuracies["accuracy"].append(accuracy)
accuracies["balanced_accuracy"].append(balanced_accuracy)
accuracies["test_accuracy"].append(test_accuracy)
accuracies["test_balanced_accuracy"].append(test_balanced_accuracy)

# now train on number of oxygen genes
print("running on number of oxygen genes")
features = ['oxygen_genes']
feature_type = "metadata"
training_data = load_training_data(feature_type=feature_type)
validation_data = load_validation_data(feature_type=feature_type)
training_data["features"] = training_data["features"][features]
validation_data["features"] = validation_data["features"][features]
cleaned_data = process_data(training_data["features"], training_data["labels"]["physiology"], validation_data["features"], validation_data["labels"]["physiology"])

model = GeneralClassifier(model_class=LogisticRegression, params={'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100000, 'C': 1})

# Fit the classifier with optimized parameters
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
# balanced accuracy
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])
#print accuracy and balanced accuracy

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])

# store accuracies in a dict for analysis later
accuracies["feature_set"].append("Number of O2 genes")
accuracies["accuracy"].append(accuracy)
accuracies["balanced_accuracy"].append(balanced_accuracy)
accuracies["test_accuracy"].append(test_accuracy)
accuracies["test_balanced_accuracy"].append(test_balanced_accuracy)



# now train on % of oxygen genes
print("running on % oxygen genes")
feature_type = "metadata"
training_data = load_training_data(feature_type=feature_type)
validation_data = load_validation_data(feature_type=feature_type)
training_data["features"]["perc_o2"] = training_data["features"]["oxygen_genes"] / training_data["features"]["number_of_annotations"]
validation_data["features"]["perc_o2"] = validation_data["features"]["oxygen_genes"] / validation_data["features"]["number_of_annotations"]
features = ['perc_o2']
training_data["features"] = training_data["features"][features]
validation_data["features"] = validation_data["features"][features]
cleaned_data = process_data(training_data["features"], training_data["labels"]["physiology"], validation_data["features"], validation_data["labels"]["physiology"])

model = GeneralClassifier(model_class=LogisticRegression, params={'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100000, 'C': 1})

# Fit the classifier with optimized parameters
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
# balanced accuracy
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])
#print accuracy and balanced accuracy


# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])


# store accuracies in a dict for analysis later
accuracies["feature_set"].append("Percent O2 genes")
accuracies["accuracy"].append(accuracy)
accuracies["balanced_accuracy"].append(balanced_accuracy)
accuracies["test_accuracy"].append(test_accuracy)
accuracies["test_balanced_accuracy"].append(test_balanced_accuracy)




# now train on gene sets, and perform hyperparameter optimization because I have multiple features now
print("running on 5 gene set")
# load gene set from assets directory, there are no columns ID or index fields
features = pd.read_csv(f"{asset_path}/gene_sets/jablonska_five_genes.csv",header=None)[0].tolist()

feature_type = "KO"
training_data = load_training_data(feature_type=feature_type)
validation_data = load_validation_data(feature_type=feature_type)
# only keep features that are columns in training and validationd data
features = [x for x in features if x in training_data["features"].columns.tolist() and x in validation_data["features"].columns.tolist()]

training_data["features"] = training_data["features"][features]
validation_data["features"] = validation_data["features"][features]
cleaned_data = process_data(training_data["features"], training_data["labels"]["physiology"], validation_data["features"], validation_data["labels"]["physiology"])

model = GeneralClassifier(model_class=LogisticRegression, params={'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100000, 'C': 1})
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}
best_params = model.hyperparameter_optimization(cleaned_data["X_train"], cleaned_data["y_train"], param_grid)

# Fit the classifier with optimized parameters
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
# balanced accuracy
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])
#print accuracy and balanced accuracy

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])

# store accuracies in a dict for analysis later
accuracies["feature_set"].append("5 gene set genes")
accuracies["accuracy"].append(accuracy)
accuracies["balanced_accuracy"].append(balanced_accuracy)
accuracies["test_accuracy"].append(test_accuracy)
accuracies["test_balanced_accuracy"].append(test_balanced_accuracy)

# now train on gene sets, and perform hyperparameter optimization because I have multiple features now
print("running on terminal oxidases gene set")
# load gene set from assets directory, there are no columns ID or index fields
features = pd.read_csv(f"{asset_path}/gene_sets/terminal_oxidase_genes.csv",header=None)[0].tolist()

feature_type = "KO"
training_data = load_training_data(feature_type=feature_type)
validation_data = load_validation_data(feature_type=feature_type)
# only keep features that are columns in training and validationd data
features = [x for x in features if x in training_data["features"].columns.tolist() and x in validation_data["features"].columns.tolist()]

training_data["features"] = training_data["features"][features]
validation_data["features"] = validation_data["features"][features]
cleaned_data = process_data(training_data["features"], training_data["labels"]["physiology"], validation_data["features"], validation_data["labels"]["physiology"])

model = GeneralClassifier(model_class=LogisticRegression, params={'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100000, 'C': 1})
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}
best_params = model.hyperparameter_optimization(cleaned_data["X_train"], cleaned_data["y_train"], param_grid)

# Fit the classifier with optimized parameters
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
# balanced accuracy
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])
#print accuracy and balanced accuracy

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])

# store accuracies in a dict for analysis later
accuracies["feature_set"].append("Terminal oxidase genes")
accuracies["accuracy"].append(accuracy)
accuracies["balanced_accuracy"].append(balanced_accuracy)
accuracies["test_accuracy"].append(test_accuracy)
accuracies["test_balanced_accuracy"].append(test_balanced_accuracy)


# now train on gene sets, and perform hyperparameter optimization because I have multiple features now
print("running on O2 associated genes")
# load gene set from assets directory, there are no columns ID or index fields
features = pd.read_csv(f"{asset_path}/gene_sets/oxygen_associated_genes.csv",header=None)[0].tolist()

feature_type = "KO"
training_data = load_training_data(feature_type=feature_type)
validation_data = load_validation_data(feature_type=feature_type)
# only keep features that are columns in training and validationd data
features = [x for x in features if x in training_data["features"].columns.tolist() and x in validation_data["features"].columns.tolist()]

training_data["features"] = training_data["features"][features]
validation_data["features"] = validation_data["features"][features]
cleaned_data = process_data(training_data["features"], training_data["labels"]["physiology"], validation_data["features"], validation_data["labels"]["physiology"])

model = GeneralClassifier(model_class=LogisticRegression, params={'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100000, 'C': 1})
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}
best_params = model.hyperparameter_optimization(cleaned_data["X_train"], cleaned_data["y_train"], param_grid)

# Fit the classifier with optimized parameters
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
# balanced accuracy
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])
#print accuracy and balanced accuracy

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])

# store accuracies in a dict for analysis later
accuracies["feature_set"].append("All O2 genes")
accuracies["accuracy"].append(accuracy)
accuracies["balanced_accuracy"].append(balanced_accuracy)
accuracies["test_accuracy"].append(test_accuracy)
accuracies["test_balanced_accuracy"].append(test_balanced_accuracy)



# now train on all KOs
print("running on KOs")
feature_type = "KO"
training_data = load_training_data(feature_type=feature_type)
validation_data = load_validation_data(feature_type=feature_type)
cleaned_data = process_data(training_data["features"], training_data["labels"]["physiology"], validation_data["features"], validation_data["labels"]["physiology"])

model = GeneralClassifier(model_class=LogisticRegression, params={'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100000, 'C': 1})
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}
best_params = model.hyperparameter_optimization(cleaned_data["X_train"], cleaned_data["y_train"], param_grid)

# Fit the classifier with optimized parameters
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
# balanced accuracy
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])
#print accuracy and balanced accuracy

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])

# store accuracies in a dict for analysis later
accuracies["feature_set"].append("All KO families")
accuracies["accuracy"].append(accuracy)
accuracies["balanced_accuracy"].append(balanced_accuracy)
accuracies["test_accuracy"].append(test_accuracy)
accuracies["test_balanced_accuracy"].append(test_balanced_accuracy)


# now run on geneome embedding vector
print("running on genome embedding vector")

feature_type = "embedding.genome"
training_data = load_training_data(feature_type=feature_type)
validation_data = load_validation_data(feature_type=feature_type)
cleaned_data = process_data(training_data["features"], training_data["labels"]["physiology"], validation_data["features"], validation_data["labels"]["physiology"])

model = GeneralClassifier(model_class=LogisticRegression, params={'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 100000, 'C': 1})
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}
best_params = model.hyperparameter_optimization(cleaned_data["X_train"], cleaned_data["y_train"], param_grid)

# Fit the classifier with optimized parameters
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
# balanced accuracy
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])
#print accuracy and balanced accuracy

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])

# store accuracies in a dict for analysis later
accuracies["feature_set"].append("All KO families")
accuracies["accuracy"].append(accuracy)
accuracies["balanced_accuracy"].append(balanced_accuracy)
accuracies["test_accuracy"].append(test_accuracy)
accuracies["test_balanced_accuracy"].append(test_balanced_accuracy)


accuracies = pd.DataFrame(accuracies)
print(accuracies)

running on number of genes
running on number of oxygen genes
running on % oxygen genes
running on 5 gene set
running on terminal oxidases gene set
running on O2 associated genes
